In [1]:
import sys, os
sys.path.insert(0, os.path.join(".."))

In [2]:
import pdfimage.pdf_read

In [3]:
pdf = pdfimage.pdf_read.PDF(os.path.join("..", "PDFReference.pdf"))
pdf.version

'%PDF-1.4'

In [4]:
pdf.trailer

PDFDictionary({PDFName(b'Size'): PDFNumeric(31667), PDFName(b'ID'): PDFArray([PDFString(b"\x19WE'\xec\xbf\x00\xe3\xec\x03s\x87\x983\xee\xf6"), PDFString(b'$\xee\x9e\xdb}\xe4\r\xb8b\xfd\xb4\xc5\xd3I5\x85')]), PDFName(b'Info'): PDFObjectId(7, 0), PDFName(b'Root'): PDFObjectId(1, 0), PDFName(b'Encrypt'): PDFObjectId(31666, 0)})

In [5]:
for key, loc in pdf.object_lookup.items():
    items = pdf.object_at(loc)
    if len(items) == 1:
        continue
    if len(items)==2 and isinstance(items[1], pdfimage.pdf_read.PDFStreamIndicator):
        obj = pdf.full_object_at(loc)
        continue
    raise AssertionError()